# Evolving Clojure through macros

### Textual substitution


In [1]:
; Changing the value of 'a-ref' with the standard 'ref-set- function

(def a-ref (ref 0))

(dosync 
    (ref-set a-ref 1))

1

In [2]:
; Changing the value of a ref with a new macro 'sync-set'

(defmacro sync-set [r v]
    (list 'dosync 
        (list 'ref-set r v)))

(sync-set a-ref 2)

2

### The 'unless' example

In [3]:
; The general form of an if condition is as follows

"""
(if test then else)
"""

""

In [4]:
; Cheching if a number is odd with the 'odd?' built-in function

(defn exhibits-oddity?-standard [x]
    (if (odd? x)
        (println "Very odd!")))


#'user/exhibits-oddity?-standard

The testing executes as expected

In [5]:
(exhibits-oddity?-standard 11)

Very odd!


nil

In [6]:
(exhibits-oddity?-standard 10)

nil

In [17]:
; Cheching if a number is odd with an user-defined function 'unless-function'

(defn unless-function [test then]
    (if (not test)
        then))

(defn exhibits-oddity?-function [x]
    (unless-function (even? x)
        (println "Very odd, indeed!")))


#'user/exhibits-oddity?-function

THe testing shows unexpected results

In [18]:
(exhibits-oddity?-function 11)

Very odd, indeed!


nil

In [19]:
; This happens since all arguments of 'exhibits-oddity?-function' 
; are evaluated before the 'if' expression in 'unless' even begins

(exhibits-oddity?-function 10)

Very odd, indeed!


nil

In [20]:
; Cheching if a number is odd with an user-defined function 'unless-function2'
; and using an anonymous function as its 2nd argument

(defn unless-function2 [test then-thunk]
    (if (not test)
        (then-thunk)))


(defn exhibits-oddity?-function2 [x]
    (unless-function2 (even? x)
        #(println "Rather odd!")))

#'user/exhibits-oddity?-function2

In [21]:
(exhibits-oddity?-function2 11)

Rather odd!


nil

In [23]:
; This works, but wrapping the argument with an anonymous function
; is not the best solution in every case

(exhibits-oddity?-function2 10)

nil

In [25]:
; Cheching if a number is odd with an user-defined macro 'unless-macro'

(defmacro unless-macro [test then]
    (list 'if (list 'not test)
        then))

(defn exhibits-oddity?-macro [x]
    (unless-macro (even? x)
        (println "Very odd, indeed!")))


#'user/exhibits-oddity?-macro

In [26]:
(exhibits-oddity?-macro 11)

Very odd, indeed!


nil

In [27]:
(exhibits-oddity?-macro 10)

nil

In [29]:
; Checking the resulting s-expression with 'macroexpand'

(macroexpand 
    '(unless (even? x) (println "Very odd, indeed!"))) 

(if (not (even? x)) (println "Very odd, indeed!"))

### Macro templates

In [1]:
; The original unless macro

(defmacro unless [test then]
    (list 'if (list 'not test)
        then))


#'user/unless

In [2]:
; Rewriting the last macro using '`' (syntax quote character) and '~' (syntax unquote character)

(defmacro unless [test then]
    `(if (not ~test)
        ~then))


#'user/unless

In [3]:
; Generalizing the macro to take an arbitrary expression as arg

(defmacro unless [test & exprs]
    `(if (not ~test)
        (do ~exprs)))


#'user/unless

In [4]:
; Using the redefined macro in the following function
; (it works, but also throws the beloved NullPointerException)

(defn exhibits-oddity? [x]
    (unless (even? x)
        (do
            (println "Odd!")
            (println "Very odd!"))))

(exhibits-oddity? 11)

Odd!
Very odd!


Execution error (NullPointerException) at user/exhibits-oddity? (REPL:5).
null


class java.lang.NullPointerException: 

In [ ]:
; Expanding the macro to see any errors

(macroexpand-1 '(unless (even? x) 
    (println "Odd!") 
    (println "Very odd!")))



(if (clojure.core/not (even? x)) (do ((println "Odd!") (println "Very odd!"))))

```
From the output, there's an extra pair of parentheses:
    do ((println "Odd!") (println "Very odd!"))
When the expected expression is:
    do (println "Odd!") (println "Very odd!")
The first expression evaluates to:
    (nil nil)
Which is interpreted as a function call, and since the argument is nil, 
the NullPointerException is thrown.
```

In [5]:
; Usint the splice reader macro ('~@') to 'unpack' a list of args
(defmacro unless [test & exprs]
    `(if (not ~test)
        (do ~@exprs)))


#'user/unless

In [7]:
; Using the redefined macro in the following function
; (it works, and the NullPointerException is removed)

(defn exhibits-oddity? [x]
    (unless (even? x)
        (do
            (println "Odd!")
            (println "Very odd!"))))

(exhibits-oddity? 11)

Odd!
Very odd!


nil

### Generating names

In [12]:
; A badly defined macro

(defmacro def-logged-fn [fn-name args & body]
    `(defn ~fn-name ~args
        (let [now (System/currentTimeMillis)]
            (println "[" now "] Call to" (str (var ~fn-name)))
            ~@body)))


#'user/def-logged-fn

In [15]:
; Calling the last macro throws a syntax error

(def-logged-fn printname [name]
    (println "hi" name))


Syntax error macroexpanding clojure.core/let at (REPL:3:1).
user/now - failed: simple-symbol? at: [:bindings :form :local-symbol] spec: :clojure.core.specs.alpha/local-name
user/now - failed: vector? at: [:bindings :form :seq-destructure] spec: :clojure.core.specs.alpha/seq-binding-form
user/now - failed: map? at: [:bindings :form :map-destructure] spec: :clojure.core.specs.alpha/map-bindings
user/now - failed: map? at: [:bindings :form :map-destructure] spec: :clojure.core.specs.alpha/map-special-binding


class clojure.lang.Compiler$CompilerException: 

In [16]:
; When using macroexpand-1, it can be seen that 'now' 
; is being redefined as 'user/now', which isn't defined

(macroexpand-1 '(def-logged-fn printname [name]
    (println "hi" name)))


(clojure.core/defn printname [name] (clojure.core/let [user/now (java.lang.System/currentTimeMillis)] (clojure.core/println "[" user/now "] Call to" (clojure.core/str (var printname))) (println "hi" name)))

In [18]:
; Rewriting the macro to use the '#' reader macro that generates
; unique names that won’t conflict with others

(defmacro def-logged-fn [fn-name args & body]
    `(defn ~fn-name ~args
        (let [now# (System/currentTimeMillis)]
            (println "[" now# "] Call to" (str (var ~fn-name)))
            ~@body)))


#'user/def-logged-fn

In [21]:
; Calling the last macro now works as expected

(def-logged-fn printname [name]
    (println "hi" name))

(printname "deepthi")

[ 1642414687208 ] Call to #'user/printname
hi deepthi


nil

## Macros from within Clojure

In [26]:
; The 'comment' macro is the simplest one: it doesn't do anything

(defmacro comment [& body])

#'user/comment

In [28]:
; The 'declare' macro takes a list of symbols and create vars named with each symbol

(defmacro declare [& names] 
    `(do ~@(map #(list 'def %) names)))

(macroexpand '(declare add multiply subtract divide))

(do (def add) (def multiply) (def subtract) (def divide))

In [30]:
; The 'defonce' macro defines a var but only once

(defmacro defonce [name expr]
    `(let [v# (def ~name)]
        (when-not (.hasRoot v#)
            (def ~name ~expr))))


#'user/defonce

In [33]:
; The 'and' macro implements the logical and
; (note the recursion at the end)

(defmacro and 
    ([] true)
    ([x] x)
    ([x & next]
        `(let [and# ~x]
            (if and# (and ~@next) and#))))

(macroexpand '(and (even? x) (> x 50) (< x 500)))

(let* [and__4261__auto__ (even? x)] (if and__4261__auto__ (user/and (> x 50) (< x 500)) and__4261__auto__))

In [36]:
; The 'time' macro measures the execution time of an expression

(defmacro time [expr]
    `(let [start# (. System (nanoTime)) ret# ~expr]
    (prn 
        (str "Elapsed time: " 
        (/ (double (- (. System (nanoTime)) start#)) 1000000.0) 
        " msecs"))
    ret#))

(time (* 1331 13531))


"Elapsed time: 0.0444 msecs"


18009761

## Writing your own macros

infix

In [2]:
; This macro allows the use of math operators with infix notation

(defmacro infix [expr]
    (let [[left op right] expr]
        (list op left right)))

(infix (2 + 2))

4

randomly

In [6]:
; This macro accepts any number of s-expressions and picks one at random.

(defmacro randomly [& exprs]
    (let [len (count exprs)
          index (rand-int len)
          conditions (map #(list '= index %) (range len))]
        `(cond ~@(interleave conditions exprs))))

(randomly (println "amit") (println "deepthi") (println "adi"))


amit


nil

In [17]:
; To see the generated s-expression of the last macro, use macroexpand-1

(macroexpand-1
    '(randomly (println "amit") (println "deepthi") (println "adi")))

(clojure.core/cond (= 1 0) (println "amit") (= 1 1) (println "deepthi") (= 1 2) (println "adi"))

defwebmethod

In [23]:
; A function that checks id an user is authenticated

(defn check-credentials [username password]
    true)

; A function that process a web request and return diferent responses
; from the user's auth status

(defn login-user [request]
    (let [username (:username request)
          password (:password request)]
        (if (check-credentials username password)
            (str "Welcome back, " username ", " password " is correct!")
            (str "Login failed!"))))

; A sample request

(def request {:username "amit" :password "123456"})


#'user/request

In [24]:
; Testing the last functions

(login-user request)


"Welcome back, amit, 123456 is correct!"

In [26]:
; A macro that abstracts the dereferencing of the user credentials

(defmacro defwebmethod [name args & exprs]
    `(defn ~name [{:keys ~args}]
        ~@exprs))

; Rewriting the login user function to use the new macro

(defwebmethod login-user [username password]
    (if (check-credentials username password)
        (str "Welcome, " username ", " password " is still correct!")
        (str "Login failed!")))


#'user/login-user

In [28]:
; Testing the last modified function

(login-user request)

"Welcome, amit, 123456 is still correct!"

defnn

In [46]:
; A macro that abstracts the destructuring of key-value pairs of a map
; And print them to the console

(defmacro defnn [fname [& names] & body]
    (let [ks {:keys (vec names)}]
        `(defn ~fname [& {:as arg-map#}]
            (let [~ks arg-map#] ~@body))))

(defnn print-details [name salary start-date]
    (println "Name:" name)
    (println "Salary:" salary)
    (println "Started on:" start-date))

(print-details :start-date "10/22/2009" :name "Rob" :salary 1000000)


Name: Rob
Salary: 1000000
Started on: 10/22/2009


nil

assert-true

In [47]:
; A macro that ckecks if a s-expression evaluates to true

(defmacro assert-true [test-expr]
    (let [[operator lhs rhs] test-expr]
        `(let [rhsv# ~rhs ret# ~test-expr]
            (if-not ret#
                (throw (RuntimeException. (str '~lhs " is not " '~operator " " rhsv#)))
                true))))


#'user/assert-true

In [48]:
; Testing with a success case

(assert-true (= (* 2 4) (/ 16 2)))


true

In [49]:
; Testing with a failure case

(assert-true (>= (* 2 4) (/ 18 2)))


Execution error at user/eval4290 (REPL:1).
(* 2 4) is not >= 9


class java.lang.RuntimeException: 

In [52]:
; Using macroexpand-1 to see the resulting s-expression

(macroexpand-1 '(assert-true (>= (* 3 3) (/ 18 2))))

(clojure.core/let [rhsv__4276__auto__ (/ 18 2) ret__4277__auto__ (>= (* 3 3) (/ 18 2))] (clojure.core/if-not ret__4277__auto__ (throw (java.lang.RuntimeException. (clojure.core/str (quote (* 3 3)) " is not " (quote >=) " " rhsv__4276__auto__))) true))

In [57]:
; A modified version of the last macro, with added error checking

(defmacro assert-true-2 [test-expr]
    (if-not (= 3 (count test-expr))
        (throw (RuntimeException. "Argument must be of the form (operator test-expr expected-expr)")))
    (if-not (some #{(first test-expr)} '(< > <= >= = not=))
        (throw (RuntimeException. "Operator must be one of < > <= >= = not=")))
    (let [[operator lhs rhs] test-expr]
        `(let [rhsv# ~rhs ret# ~test-expr]
            (if-not ret# (throw (RuntimeException. (str '~lhs " is not " '~operator " " rhsv#)))
            true))))


#'user/assert-true-2

In [58]:
; Catching the 'Argument must be of the...' error

(assert-true-2 (>= (* 2 4) (/ 18 2) (+ 2 5)))

Unexpected error macroexpanding assert-true-2 at (REPL:3:1).
Argument must be of the form (operator test-expr expected-expr)


class clojure.lang.Compiler$CompilerException: 

In [59]:
; Catching the 'Operator must be one...' error

(assert-true-2 (<> (* 2 4) (/ 16 2)))


Unexpected error macroexpanding assert-true-2 at (REPL:3:1).
Operator must be one of < > <= >= = not=


class clojure.lang.Compiler$CompilerException: 